# Livro Simpy - Simulação Discreta

## Link de Referencia
* https://simpy.livrosimulacao.eng.br/
* https://simpy.readthedocs.io/en/latest/contents.html

## Exercício de prática 1

In [36]:
import random             # gerador de números aleatórios
import simpy              # biblioteca de simulação

def distributions(tipo):
    TEMPO_MEDIO_CHEGADAS = 1.0          # tempo médio entre chegadas sucessivas de clientes
    TEMPO_MEDIO_ATENDIMENTO = 0.5       # tempo médio de atendimento no servidor
    
    distributions = {
        'chegada_cliente': random.expovariate(1.0/TEMPO_MEDIO_CHEGADAS),
        'atendimento_cliente': random.expovariate(1.0/TEMPO_MEDIO_ATENDIMENTO)
    }
    return distributions[tipo]


def geraChegadas(env):
    # função que cria chegadas de entidades no sistema
    contaChegada = 0
    while True:
        yield env.timeout(distributions('chegada_cliente'))
        contaChegada += 1
        print(f"Cliente {contaChegada} chega em: {env.now}")
        env.process(atendimentoServidor(env, contaChegada, servidorRes))


def atendimentoServidor(env, nome, servidorResource):
    request = servidorResource.request()
    tempo_chegada = env.now
    
    yield request
    print(f'Servidor inicia o atendimento do Cliente {nome} as: {env.now}')
    print(f'Tempo de permanência do Cliente {nome} na fila: {env.now - tempo_chegada}')
    yield env.timeout(distributions('atendimento_cliente'))
    print(f'Servidor termina o atendimento do Cliente {nome} as: {env.now}, Fila atual: {len(servidorRes.queue)}')
    yield servidorResource.release(request)
        
random.seed(1000)         # semente do gerador de números aleatórios
env = simpy.Environment() # cria o environment do modelo
servidorRes = simpy.Resource(env, capacity=1)
env.process(geraChegadas(env)) # cria o processo de chegadas
env.run(until=25) # roda a simulação por 10 unidades de tempo

Cliente 1 chega em: 1.5021840823801902
Servidor inicia o atendimento do Cliente 1 as: 1.5021840823801902
Tempo de permanência do Cliente 1 na fila: 0.0
Cliente 2 chega em: 1.6065890590717269
Servidor termina o atendimento do Cliente 1 as: 1.884703071733432, Fila atual: 1
Servidor inicia o atendimento do Cliente 2 as: 1.884703071733432
Tempo de permanência do Cliente 2 na fila: 0.2781140126617052
Servidor termina o atendimento do Cliente 2 as: 2.111161820540629, Fila atual: 0
Cliente 3 chega em: 5.437449718273912
Servidor inicia o atendimento do Cliente 3 as: 5.437449718273912
Tempo de permanência do Cliente 3 na fila: 0.0
Servidor termina o atendimento do Cliente 3 as: 5.566616886332236, Fila atual: 0
Cliente 4 chega em: 6.108513946323223
Servidor inicia o atendimento do Cliente 4 as: 6.108513946323223
Tempo de permanência do Cliente 4 na fila: 0.0
Servidor termina o atendimento do Cliente 4 as: 6.442717888676174, Fila atual: 0
Cliente 5 chega em: 6.721121419589968
Servidor inicia o at

## Exercício de prática 2 - Lavanderia

In [42]:
import random
import simpy

contaClientes=0             # conta clientes que chegaram no sistema

def distributions(tipo):
    # função que armazena as distribuições utilizadas no modelo
    return {
        'chegadas': random.expovariate(1.0/5.0),
        'lavar': 20,
        'descarregar': random.uniform(1, 4),
        'carregar': 2,
        'secar': random.uniform(9, 12),
    }.get(tipo, 0.0)

def chegadaClientes(env, lavadoras, cestos, secadoras):
    # função que gera a chegada de clientes
    global contaClientes
    while True:
        yield env.timeout(distributions('chegadas'))
        contaPecas += 1
        print(f"Cliente {contaPecas} chega em: {env.now}")
        

    # chamada do processo de lavagem e secagem
        env.process(lavaSeca(env, contaClientes, lavadoras, cestos, secadoras))

def lavaSeca(env, cliente, lavadoras, cestos, secadoras):
    # função que processa a operação de cada cliente dentro da lavanderia
    
    
    # ocupa a lavadora
    req_lavadora = lavadoras.request()
    yield req_lavadora
    print(f'Lavadora inicia a lavagem do Cliente {cliente} as: {env.now}')
    yield env.timeout(distributions('lavar'))
    
    # antes de retirar da lavadora, pega um cesto
    req_cesto = cestos.request()
    yield req_cesto
    print(f'Cliente {cliente} ocupa o cesto as: {env.now}')
    yield env.timeout(distributions('descarregar'))
    

    # libera a lavadora, mas não o cesto
    yield lavadoras.release(req_lavadora)
    print(f'Cliente {cliente} desocupa a lavadora as: {env.now}')

    # ocupa a secadora antes de liberar o cesto
    req_secadora = secadoras.request()
    yield req_secadora
    print(f'Cliente {cliente} ocupa a secadora as: {env.now}')
    yield env.timeout(distributions('carregar'))
    
    # libera o cesto mas não a secadora
    yield cestos.release(req_cesto)
    print(f'Cliente {cliente} desocupa o cesto as: {env.now}')
    yield env.timeout(distributions('secar'))
    
    # pode liberar a secadora
    print(f'Cliente {cliente} iniciar o descarregamento as: {env.now}')
    yield env.timeout(distributions('descarregar'))	
    
    yield secadoras.release(req_secadora)
    print(f'Cliente {cliente} desocupa o secadora as: {env.now}')
    

random.seed(10)
env = simpy.Environment()
lavadoras = simpy.Resource(env, capacity=3)
cestos = simpy.Resource(env, capacity=2)
secadoras = simpy.Resource(env, capacity=1)
env.process(chegadaClientes(env, lavadoras, cestos, secadoras))

env.run(until=40)

Cliente 1 chega em: 4.236186249169292
Lavadora inicia a lavagem do Cliente 1 as: 4.236186249169292
Cliente 2 chega em: 5.390163966411981
Lavadora inicia a lavagem do Cliente 2 as: 5.390163966411981
Cliente 3 chega em: 7.375958554667703
Lavadora inicia a lavagem do Cliente 3 as: 7.375958554667703
Cliente 4 chega em: 11.997454764551819
Cliente 5 chega em: 17.418728826706193
Cliente 1 ocupa o cesto as: 24.236186249169293
Cliente 1 desocupa a lavadora as: 25.248351681644515
Lavadora inicia a lavagem do Cliente 4 as: 25.248351681644515
Cliente 1 ocupa a secadora as: 25.248351681644515
Cliente 2 ocupa o cesto as: 25.390163966411983
Cliente 2 desocupa a lavadora as: 26.585146494792536
Lavadora inicia a lavagem do Cliente 5 as: 26.585146494792536
Cliente 1 desocupa o cesto as: 27.248351681644515
Cliente 3 ocupa o cesto as: 27.375958554667704
Cliente 3 desocupa a lavadora as: 30.349631370945033
Cliente 6 chega em: 37.62177605245362
Lavadora inicia a lavagem do Cliente 6 as: 37.62177605245362
Cl

# Lista de Exercícios 1 - Simulação Discreta

## Lavanderia

In [50]:
import random
import simpy

contaClientes = 0             # conta clientes que chegaram no sistema

def distributions(tipo):
    # função que armazena as distribuições utilizadas no modelo
    return {
        'chegadas': random.expovariate(1.0/10.0),
        'lavar': 25,
        'descarregar': random.uniform(1, 4),
        'carregar': 2,
        'transportar': random.uniform(3,5),
        'secar': random.uniform(9, 12),
    }.get(tipo, 0.0)

def chegadaClientes(env, lavadoras, cestos, secadoras):
    # função que gera a chegada de clientes
    global contaClientes
    while True:
        yield env.timeout(distributions('chegadas'))
        contaClientes += 1
        print(f"Cliente {contaClientes} chega em: {env.now}")
        

    # chamada do processo de lavagem e secagem
        env.process(lavaSeca(env, contaClientes, lavadoras, cestos, secadoras))

def lavaSeca(env, cliente, lavadoras, cestos, secadoras):
    # função que processa a operação de cada cliente dentro da lavanderia
    
    
    # ocupa a lavadora
    req_lavadora = lavadoras.request()
    yield req_lavadora
    print(f'Lavadora inicia a lavagem do Cliente {cliente} as: {env.now}')
    yield env.timeout(distributions('lavar'))
    
    # antes de retirar da lavadora, pega um cesto
    req_cesto = cestos.request()
    yield req_cesto
    print(f'Cliente {cliente} ocupa o cesto as: {env.now}')
    print(f'Cliente {cliente} inicia o descarregamento da maquina as: {env.now}')
    yield env.timeout(distributions('descarregar'))
    

    # libera a lavadora, mas não o cesto
    yield lavadoras.release(req_lavadora)
    print(f'Cliente {cliente} desocupa a lavadora as: {env.now}')
    
    # cliente trasporta o cesto ate a maquina de secar
    print(f'Cliente inicia o transporte das roupas para a máquina de secar as: {env.now}')
    yield env.timeout(distributions('transportar'))
    print(f'Cliente terminar o transporte das roupas para a máquina de secar as: {env.now}')
    
    # ocupa a secadora antes de liberar o cesto
    req_secadora = secadoras.request()
    yield req_secadora
    print(f'Cliente {cliente} ocupa a secadora as: {env.now}')
    yield env.timeout(distributions('carregar'))
    
    # libera o cesto mas não a secadora
    yield cestos.release(req_cesto)
    print(f'Cliente {cliente} desocupa o cesto as: {env.now}')
    yield env.timeout(distributions('secar'))
    
    # pode liberar a secadora
    print(f'Cliente {cliente} iniciar o descarregamento da secadora as: {env.now}')
    yield env.timeout(distributions('descarregar'))	
    
    yield secadoras.release(req_secadora)
    print(f'Cliente {cliente} desocupa o secadora as: {env.now}')
    

random.seed(10)
env = simpy.Environment()
lavadoras = simpy.Resource(env, capacity=7)
cestos = simpy.Resource(env, capacity=12)
secadoras = simpy.Resource(env, capacity=1)
env.process(chegadaClientes(env, lavadoras, cestos, secadoras))

env.run(until=600)

print("Fila de clientes ao final da simulação: lavadoras %i cestos %i secadoras %i" 
        %(len(lavadoras.queue), len(cestos.queue), len(secadoras.queue)))

Cliente 1 chega em: 8.472372498338585
Lavadora inicia a lavagem do Cliente 1 as: 8.472372498338585
Cliente 2 chega em: 25.256033109779743
Lavadora inicia a lavagem do Cliente 2 as: 25.256033109779743
Cliente 1 ocupa o cesto as: 33.472372498338586
Cliente 1 inicia o descarregamento da maquina as: 33.472372498338586
Cliente 1 desocupa a lavadora as: 36.45791145849564
Cliente inicia o transporte das roupas para a máquina de secar as: 36.45791145849564
Cliente terminar o transporte das roupas para a máquina de secar as: 40.68456509958906
Cliente 1 ocupa a secadora as: 40.68456509958906
Cliente 1 desocupa o cesto as: 42.68456509958906
Cliente 2 ocupa o cesto as: 50.25603310977974
Cliente 2 inicia o descarregamento da maquina as: 50.25603310977974
Cliente 2 desocupa a lavadora as: 51.45101563816029
Cliente inicia o transporte das roupas para a máquina de secar as: 51.45101563816029
Cliente 1 iniciar o descarregamento da secadora as: 53.33151165162627
Cliente terminar o transporte das roupas 

## Cliníca Médica

In [2]:
import random
import simpy

contaClientes = 0             # conta clientes que chegaram no sistema

def distributions(tipo):
    # função que armazena as distribuições utilizadas no modelo
    return {
        'chegadas': random.expovariate(1.0/3.0),
        'preenchimentos': random.expovariate(1.0/10.0),
        'atendimento': random.expovariate(1.0/20.0),
        'pagamentos': random.uniform(1,4)
    }.get(tipo, 0.0)

def chegadaClientes(env, medicos, recepcionistas):
    # função que gera a chegada de clientes
    global contaClientes
    while True:
        yield env.timeout(distributions('chegadas'))
        contaClientes += 1
        print(f"Cliente {contaClientes} chega em: {env.now}")
    
    # chamada do processo de atendimentos
        env.process(atendeConsulta(env, contaClientes, medicos, recepcionistas))

def atendeConsulta(env, cliente, medicos, recepcionistas):
    # função que processa a operação de cada cliente dentro da lavanderia
    
    
    # ocupa a recepcionista
    print("Fila de clientes ao atual: medicos %i recepcionistas %i" 
        %(len(medicos.queue), len(recepcionistas.queue)))
    
    req_recepcionista = recepcionistas.request()
    yield req_recepcionista
    print(f'Recepcionista inicia preenchimentos do Cliente {cliente} as: {env.now}')
    yield env.timeout(distributions('preenchimentos'))
    print(f'Recepcionista termina preenchimentos do Cliente {cliente} as: {env.now}')
    yield recepcionistas.release(req_recepcionista)
    
    
    # ocupa o medico
    req_medicos = medicos.request()
    yield req_medicos
    print(f'Medico inicia preenchimentos do Cliente {cliente} as: {env.now}')
    yield env.timeout(distributions('atendimento'))
    print(f'Medico termina preenchimentos do Cliente {cliente} as: {env.now}')
    yield medicos.release(req_medicos)

    #ocupa a recepcionista para o pagamento
    req_recepcionista = recepcionistas.request()
    print(f'Recepcionista inicia pagamento do Cliente {cliente} as: {env.now}')
    yield env.timeout(distributions('preenchimentos'))
    print(f'Recepcionista termina pagamento do Cliente {cliente} as: {env.now}')
    yield recepcionistas.release(req_recepcionista)
    
    

random.seed(10)
env = simpy.Environment()
#
medicos = simpy.Resource(env, capacity=3)
recepcionistas = simpy.Resource(env, capacity=2)

#mundo ideal :)
# medicos = simpy.Resource(env, capacity=8)
# recepcionistas = simpy.Resource(env, capacity=13)
env.process(chegadaClientes(env, medicos, recepcionistas))

env.run(until=600)

print("Fila de clientes ao final da simulação: medicos %i recepcionistas %i" 
        %(len(medicos.queue), len(recepcionistas.queue)))

Cliente 1 chega em: 2.5417117495015757
Fila de clientes ao atual: medicos 0 recepcionistas 0
Recepcionista inicia preenchimentos do Cliente 1 as: 2.5417117495015757
Recepcionista termina preenchimentos do Cliente 1 as: 6.51330092601302
Medico inicia preenchimentos do Cliente 1 as: 6.51330092601302
Cliente 2 chega em: 7.576809932933924
Fila de clientes ao atual: medicos 0 recepcionistas 0
Recepcionista inicia preenchimentos do Cliente 2 as: 7.576809932933924
Cliente 3 chega em: 9.018698317265528
Fila de clientes ao atual: medicos 0 recepcionistas 0
Recepcionista inicia preenchimentos do Cliente 3 as: 9.018698317265528
Recepcionista termina preenchimentos do Cliente 3 as: 10.457084786877097
Medico inicia preenchimentos do Cliente 3 as: 10.457084786877097
Medico termina preenchimentos do Cliente 3 as: 18.00524718397107
Recepcionista inicia pagamento do Cliente 3 as: 18.00524718397107
Recepcionista termina preenchimentos do Cliente 2 as: 18.419358057242675
Medico inicia preenchimentos do C

In [1]:
pip install simpy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import random
import simpy



def distributions(tipo):
    # função que armazena as distribuições utilizadas no modelo
    return {
        'fixacao': random.normalvariate((2/3), (1/12)),
        'usinagem': 3,
        'uniao': random.uniform(3.5, 4),
        'abastecimento': random.normalvariate(40,3),
        'parafusos': random.uniform(990,1010),
        'pecas': 60
    }.get(tipo, 0.0)



def montagem(env, funcionario_fix_usi, funcionario_une, pecas, parafusos):
    while True:

        
        
        
        funcionario_fix_usi_req = funcionario_fix_usi.request()
        yield funcionario_fix_usi_req
        print(f'Funcionario inicia fixacao as: {env.now}')

        yield pecas.get(2)
        yield env.timeout(distributions('fixacao'))

        print(f'Funcionario finaliza fixacao as : {env.now}')
        yield funcionario_fix_usi.release(funcionario_fix_usi_req)

        yield funcionario_fix_usi_req
        print(f'Funcionario inicia usinagem as : {env.now}')
        yield env.timeout(distributions('usinagem'))
        print(f'Funcionario finaliza usinagem as : {env.now}')
        yield funcionario_fix_usi.release(funcionario_fix_usi_req)

        funcionario_une_req = funcionario_une.request()
        yield funcionario_une_req
        print(f'Funcionario inicia uniao as: {env.now}')

        yield parafusos.get(4)
        
        yield env.timeout(distributions('uniao'))
        print(f'Funcionario finaliza uniao as : {env.now}')
        yield funcionario_une.release(funcionario_une_req)

def estoque(env, pecas, parafusos):
    while True:
        print(f'Inicia processo de abastecimento as: {env.now}')
        yield env.timeout(distributions('abastecimento'))
        print(f'Termina processo de abastecimento as: {env.now}')
        qtd_pecas = distributions('pecas')
        qtd_parafusos = distributions('parafusos')
        yield parafusos.put(qtd_parafusos)
        yield pecas.put(qtd_pecas)
        print(f'O estoque foi reabastecido com: parafusos {qtd_parafusos}  pecas {qtd_pecas}')
        print(f'Atual nível do estoque: parafusos {parafusos.level}  pecas {pecas.level}')

    

random.seed(10)
env = simpy.Environment()
parafusos = simpy.Container(env, init=990)
pecas = simpy.Container(env, init=60)
funcionario_fix_usi = simpy.Resource(env, capacity=1)
funcionario_une = simpy.Resource(env, capacity=1)
env.process(montagem(env, funcionario_fix_usi, funcionario_une, pecas, parafusos))
env.process(estoque(env, parafusos=parafusos, pecas=pecas))

env.run(until=600)

print("Fila de clientes ao final da simulação: medicos %i recepcionistas %i" 
        %(len(medicos.queue), len(recepcionistas.queue)))

Inicia processo de abastecimento as: 0
Funcionario inicia fixacao as: 0
Funcionario finaliza fixacao as : 0.6710623606946468
Funcionario inicia usinagem as : 0.6710623606946468
Funcionario finaliza usinagem as : 3.6710623606946466
Funcionario inicia uniao as: 3.6710623606946466
Funcionario finaliza uniao as : 7.17308993277385
Funcionario inicia fixacao as: 7.17308993277385
Funcionario finaliza fixacao as : 7.780914864810733
Funcionario inicia usinagem as : 7.780914864810733
Funcionario finaliza usinagem as : 10.780914864810732
Funcionario inicia uniao as: 10.780914864810732
Funcionario finaliza uniao as : 14.708477735197533
Funcionario inicia fixacao as: 14.708477735197533
Funcionario finaliza fixacao as : 15.391470430495668
Funcionario inicia usinagem as : 15.391470430495668
Funcionario finaliza usinagem as : 18.391470430495666
Funcionario inicia uniao as: 18.391470430495666
Funcionario finaliza uniao as : 22.119337859836186
Funcionario inicia fixacao as: 22.119337859836186
Funcionari